In [14]:
# ================================================
# 02_feature_engineering.ipynb
# Feature generation for NBA game prediction model
# ================================================

In [15]:
import pandas as pd
import numpy as np
from pathlib import Path

# Set paths
DATA_RAW = Path("../data/raw")
DATA_FINAL = Path("../data/final")

# Display all columns for debugging
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 180)

In [ ]:
# ===============================================================
# 1. Load cleaned datasets
# ===============================================================

games = pd.read_csv(DATA_RAW / "Games.csv", low_memory=False)
team_stats = pd.read_csv(DATA_RAW / "TeamStatistics.csv", low_memory=False)
player_stats = pd.read_csv(DATA_RAW / "PlayerStatistics.csv", low_memory=False)

# Convert dates
for df, col in [(games, "gameDate"), (team_stats, "gameDate"), (player_stats, "gameDate")]:
    df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)

print(f"Loaded datasets | Games: {games.shape}, TeamStats: {team_stats.shape}, PlayerStats: {player_stats.shape}")

✅ Loaded datasets | Games: (72008, 17), TeamStats: (144016, 48), PlayerStats: (1631540, 35)


In [78]:
# ===============================================================
# 2. Aggregate player statistics to team level
# ===============================================================

# Detect correct gameId column automatically
possible_gameid_cols = [c for c in player_stats.columns if "gameid" in c.lower()]
gameid_col = possible_gameid_cols[0] if possible_gameid_cols else None
if not gameid_col:
    raise KeyError("No 'gameId' column found in player_stats")

# Aggregate player-level stats per team per game
player_team_agg = (
    player_stats.groupby([gameid_col, "playerteamName"])
    .agg({
        "points": "sum",
        "assists": "sum",
        "reboundsTotal": "sum",
        "steals": "sum",
        "blocks": "sum",
        "numMinutes": "mean",
        "fieldGoalsPercentage": "mean",
        "threePointersPercentage": "mean",
        "freeThrowsPercentage": "mean",
    })
    .reset_index()
    .rename(columns={gameid_col: "gameId", "playerteamName": "teamName"})
)

print(f"Aggregated player stats: {player_team_agg.shape}")
display(player_team_agg.head(3))

Aggregated player stats: (144012, 11)


,gameId,teamName,points,assists,reboundsTotal,steals,blocks,numMinutes,fieldGoalsPercentage,threePointersPercentage,freeThrowsPercentage
0,10300001,Jazz,90.0,23.0,41.0,8.0,4.0,5.0,0.513474,0.052632,0.430579
1,10300001,Mavericks,85.0,20.0,38.0,9.0,4.0,5.0,0.379182,0.090909,0.290000
2,10300002,Bucks,94.0,20.0,43.0,9.0,4.0,5.0,0.347462,0.038462,0.491538


In [79]:
# ===============================================================
# 3. Standardize key columns in team_stats before merge
# ===============================================================

# Strip accidental spaces
team_stats.columns = team_stats.columns.str.strip()

# Auto-detect and rename gameId column
possible_gameid_cols = [c for c in team_stats.columns if "gameid" in c.lower()]
if possible_gameid_cols:
    old_col = possible_gameid_cols[0]
    team_stats.rename(columns={old_col: "gameId"}, inplace=True)
else:
    raise KeyError("No 'gameId' column found in team_stats!")

# Auto-detect and rename teamName column
possible_teamname_cols = [c for c in team_stats.columns if "teamname" in c.lower()]
if possible_teamname_cols:
    old_col = possible_teamname_cols[0]
    team_stats.rename(columns={old_col: "teamName"}, inplace=True)
else:
    raise KeyError("No 'teamName' column found in team_stats!")

print("Standardized key columns in team_stats:")
display(team_stats[["gameId", "teamName"]].head())


Standardized key columns in team_stats:


,gameId,teamName
0,22500127,Warriors
1,22500127,Clippers
2,22500125,Bucks
3,22500125,Knicks
4,22500126,Kings


In [80]:
# ===============================================================
# 4. Merge aggregated player stats with team stats
# ===============================================================

# Merge team-level official stats with aggregated player-level stats
merged_team = pd.merge(
    team_stats,
    player_team_agg,
    how="left",
    on=["gameId", "teamName"],
    suffixes=("", "_players")
)

# Remove duplicates
merged_team.drop_duplicates(subset=["gameId", "teamName"], inplace=True)

# Sanity checks
missing_teams = merged_team["gameId"].nunique() - team_stats["gameId"].nunique()
if missing_teams == 0:
    print(f"Merged dataset OK: {merged_team.shape}")
else:
    print(f"{abs(missing_teams)} games may be missing after merge.")

# Validation checks
print("\nNull values per key column:")
print(merged_team[["gameId", "teamName"]].isna().sum())

print("\nDuplicated gameId–teamName pairs:", 
      merged_team.duplicated(subset=["gameId", "teamName"]).sum())

display(merged_team.head(3))

Merged dataset OK: (144016, 57)

Null values per key column:
gameId      0
teamName    0
dtype: int64

Duplicated gameId–teamName pairs: 0


,gameId,gameDate,teamCity,teamName,teamId,opponentTeamCity,opponentTeamName,opponentTeamId,home,win,teamScore,opponentScore,assists,blocks,steals,fieldGoalsAttempted,fieldGoalsMade,fieldGoalsPercentage,threePointersAttempted,threePointersMade,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,numMinutes,q1Points,q2Points,q3Points,q4Points,benchPoints,biggestLead,biggestScoringRun,leadChanges,pointsFastBreak,pointsFromTurnovers,pointsInThePaint,pointsSecondChance,timesTied,timeoutsRemaining,seasonWins,seasonLosses,coachId,points,assists_players,reboundsTotal_players,steals_players,blocks_players,numMinutes_players,fieldGoalsPercentage_players,threePointersPercentage_players,freeThrowsPercentage_players
0,22500127,2025-10-28 23:00:00+00:00,Golden State,Warriors,1610612744,LA,Clippers,1610612746,1,1,98,79,24.0,3.0,11.0,86.0,39.0,0.453,45.0,17.0,0.378,4.0,3.0,0.750,32.0,10.0,42.0,15.0,13.0,19.0,240.0,33.0,13.0,32.0,20.0,30.0,24.0,10.0,2.0,17.0,14.0,42.0,12.0,3.0,1.0,4.0,1.0,NaN,98.0,24.0,42.0,11.0,3.0,16.915000,0.332429,0.261857,0.071429
1,22500127,2025-10-28 23:00:00+00:00,LA,Clippers,1610612746,Golden State,Warriors,1610612744,0,0,79,98,10.0,2.0,7.0,82.0,30.0,0.366,33.0,6.0,0.182,16.0,13.0,0.813,34.0,13.0,47.0,5.0,13.0,-19.0,240.0,25.0,24.0,14.0,16.0,22.0,5.0,9.0,2.0,9.0,8.0,42.0,19.0,3.0,0.0,2.0,2.0,NaN,79.0,10.0,47.0,7.0,2.0,16.914286,0.310357,0.133357,0.250000
2,22500125,2025-10-28 20:00:00+00:00,Milwaukee,Bucks,1610612749,New York,Knicks,1610612752,1,1,121,111,27.0,8.0,7.0,79.0,43.0,0.544,34.0,14.0,0.412,24.0,21.0,0.875,34.0,4.0,38.0,21.0,13.0,10.0,240.0,27.0,32.0,35.0,27.0,34.0,10.0,10.0,15.0,21.0,19.0,50.0,6.0,12.0,1.0,3.0,1.0,NaN,121.0,27.0,38.0,7.0,8.0,23.840000,0.340571,0.258357,0.396214


In [81]:
# ===============================================================
# 5. Proceed to compute feature differentials
# ===============================================================

print("\nReady for differential feature computation.")
print(f"Total unique gameIds: {merged_team['gameId'].nunique()}")


Ready for differential feature computation.
Total unique gameIds: 72008


In [ ]:
# ===============================================================
# 6. Compute feature differentials (home – away)
# ===============================================================

def diff_features(df, prefix_home, prefix_away, columns):
    """
    Compute (home - away) feature differences for the selected columns.
    Returns a DataFrame with 'diff_' prefixed columns.
    """
    diff_df = pd.DataFrame()
    for col in columns:
        home_col = f"{prefix_home}_{col}"
        away_col = f"{prefix_away}_{col}"
        if home_col in df.columns and away_col in df.columns:
            diff_df[f"diff_{col}"] = df[home_col] - df[away_col]
        else:
            print(f"Missing columns for: {col}")
    return diff_df


# --- Split dataset into home and away subsets ---
home = merged_team[merged_team["home"] == 1].add_prefix("home_")
away = merged_team[merged_team["home"] == 0].add_prefix("away_")

# --- Merge home and away by gameId ---
combined = pd.merge(
    home,
    away,
    left_on="home_gameId",
    right_on="away_gameId",
    how="inner",
    suffixes=("_home", "_away")
)

# --- Compute statistical differentials (team + player aggregates) ---
stats_cols = [
    "points", "assists", "reboundsTotal", "steals", "blocks",
    "fieldGoalsPercentage", "threePointersPercentage", "freeThrowsPercentage",
    "assists_players", "reboundsTotal_players", "steals_players", "blocks_players"
]

diffs = diff_features(combined, "home", "away", stats_cols)

# Combine base game info + differentials
final_df = pd.concat([combined, diffs], axis=1)

# --- Integrity check ---
expected_games = merged_team["gameId"].nunique()
actual_games = final_df["home_gameId"].nunique()
if expected_games == actual_games:
    print(f"Feature differential dataset created successfully.")
else:
    print(f"Mismatch: expected {expected_games}, found {actual_games} after merge.")

print(f"Shape: {final_df.shape}")
display(final_df.head(3))

# --- Cleanup redundant columns ---
final_df.drop(columns=["away_gameId", "away_gameDate"], inplace=True, errors="ignore")
print("Cleaned redundant columns: ['away_gameId', 'away_gameDate']")

Feature differential dataset created successfully.
Shape: (72008, 126)


,home_gameId,home_gameDate,home_teamCity,home_teamName,home_teamId,home_opponentTeamCity,home_opponentTeamName,home_opponentTeamId,home_home,home_win,home_teamScore,home_opponentScore,home_assists,home_blocks,home_steals,home_fieldGoalsAttempted,home_fieldGoalsMade,home_fieldGoalsPercentage,home_threePointersAttempted,home_threePointersMade,home_threePointersPercentage,home_freeThrowsAttempted,home_freeThrowsMade,home_freeThrowsPercentage,home_reboundsDefensive,home_reboundsOffensive,home_reboundsTotal,home_foulsPersonal,home_turnovers,home_plusMinusPoints,home_numMinutes,home_q1Points,home_q2Points,home_q3Points,home_q4Points,home_benchPoints,home_biggestLead,home_biggestScoringRun,home_leadChanges,home_pointsFastBreak,home_pointsFromTurnovers,home_pointsInThePaint,home_pointsSecondChance,home_timesTied,home_timeoutsRemaining,home_seasonWins,home_seasonLosses,home_coachId,home_points,home_assists_players,home_reboundsTotal_players,home_steals_players,home_blocks_players,home_numMinutes_players,home_fieldGoalsPercentage_players,home_threePointersPercentage_players,home_freeThrowsPercentage_players,away_gameId,away_gameDate,away_teamCity,away_teamName,away_teamId,away_opponentTeamCity,away_opponentTeamName,away_opponentTeamId,away_home,away_win,away_teamScore,away_opponentScore,away_assists,away_blocks,away_steals,away_fieldGoalsAttempted,away_fieldGoalsMade,away_fieldGoalsPercentage,away_threePointersAttempted,away_threePointersMade,away_threePointersPercentage,away_freeThrowsAttempted,away_freeThrowsMade,away_freeThrowsPercentage,away_reboundsDefensive,away_reboundsOffensive,away_reboundsTotal,away_foulsPersonal,away_turnovers,away_plusMinusPoints,away_numMinutes,away_q1Points,away_q2Points,away_q3Points,away_q4Points,away_benchPoints,away_biggestLead,away_biggestScoringRun,away_leadChanges,away_pointsFastBreak,away_pointsFromTurnovers,away_pointsInThePaint,away_pointsSecondChance,away_timesTied,away_timeoutsRemaining,away_seasonWins,away_seasonLosses,away_coachId,away_points,away_assists_players,away_reboundsTotal_players,away_steals_players,away_blocks_players,away_numMinutes_players,away_fieldGoalsPercentage_players,away_threePointersPercentage_players,away_freeThrowsPercentage_players,diff_points,diff_assists,diff_reboundsTotal,diff_steals,diff_blocks,diff_fieldGoalsPercentage,diff_threePointersPercentage,diff_freeThrowsPercentage,diff_assists_players,diff_reboundsTotal_players,diff_steals_players,diff_blocks_players
0,22500127,2025-10-28 23:00:00+00:00,Golden State,Warriors,1610612744,LA,Clippers,1610612746,1,1,98,79,24.0,3.0,11.0,86.0,39.0,0.453,45.0,17.0,0.378,4.0,3.0,0.750,32.0,10.0,42.0,15.0,13.0,19.0,240.0,33.0,13.0,32.0,20.0,30.0,24.0,10.0,2.0,17.0,14.0,42.0,12.0,3.0,1.0,4.0,1.0,NaN,98.0,24.0,42.0,11.0,3.0,16.915000,0.332429,0.261857,0.071429,22500127,2025-10-28 23:00:00+00:00,LA,Clippers,1610612746,Golden State,Warriors,1610612744,0,0,79,98,10.0,2.0,7.0,82.0,30.0,0.366,33.0,6.0,0.182,16.0,13.0,0.813,34.0,13.0,47.0,5.0,13.0,-19.0,240.0,25.0,24.0,14.0,16.0,22.0,5.0,9.0,2.0,9.0,8.0,42.0,19.0,3.0,0.0,2.0,2.0,NaN,79.0,10.0,47.0,7.0,2.0,16.914286,0.310357,0.133357,0.250000,19.0,14.0,-5.0,4.0,1.0,0.087,0.196,-0.063,14.0,-5.0,4.0,1.0
1,22500125,2025-10-28 20:00:00+00:00,Milwaukee,Bucks,1610612749,New York,Knicks,1610612752,1,1,121,111,27.0,8.0,7.0,79.0,43.0,0.544,34.0,14.0,0.412,24.0,21.0,0.875,34.0,4.0,38.0,21.0,13.0,10.0,240.0,27.0,32.0,35.0,27.0,34.0,10.0,10.0,15.0,21.0,19.0,50.0,6.0,12.0,1.0,3.0,1.0,NaN,121.0,27.0,38.0,7.0,8.0,23.840000,0.340571,0.258357,0.396214,22500125,2025-10-28 20:00:00+00:00,New York,Knicks,1610612752,Milwaukee,Bucks,1610612749,0,0,111,121,24.0,2.0,9.0,92.0,39.0,0.424,40.0,13.0,0.325,23.0,20.0,0.870,28.0,17.0,45.0,21.0,13.0,-10.0,240.0,26.0,45.0,20.0,20.0,18.0,14.0,13.0,15.0,9.0,13.0,42.0,10.0,12.0,1.0,2.0,2.0,NaN,111.0,24.0,45.0,9.0,2.0,23.840000,0.240462,0.191538,0.341923,10.0,3.0,-7.0,-2.0,6.0,0.120,0.087,0.005,3.0,-7.0,-2.0,6.0
2,22500126,2025-10-28 20:00:00+00:00,Oklahoma City,Thunder,16106

🧹 Cleaned redundant columns: ['away_gameId', 'away_gameDate']


In [84]:
# ===============================================================
# 7. Target variables
# ===============================================================

# Define binary win target and continuous score differential
final_df["home_win"] = (final_df["home_points"] > final_df["away_points"]).astype(int)
final_df["score_diff"] = final_df["home_points"] - final_df["away_points"]

print("Target variables created successfully.")
display(final_df[["home_points", "away_points", "home_win", "score_diff"]].head())

Target variables created successfully.


,home_points,away_points,home_win,score_diff
0,98.0,79.0,1,19.0
1,121.0,111.0,1,10.0
2,107.0,101.0,1,6.0
3,144.0,117.0,1,27.0
4,134.0,139.0,0,-5.0


In [86]:
# ===============================================================
# 8. Add contextual and categorical features
# ===============================================================
# Adds contextual metadata about game type and environment.
# Handles cases where 'gameType' column may have variant names (e.g., 'home_gameType').

# --- 1. Detect home game type column dynamically ---
game_type_cols = [c for c in final_df.columns if "gameType" in c and "home" in c]

if game_type_cols:
    col = game_type_cols[0]
    print(f"📂 Detected game type column: {col}")
    
    # Create categorical flags
    final_df["is_playoffs"] = (final_df[col].str.contains("Playoff", case=False, na=False)).astype(int)
    final_df["is_regular"] = (final_df[col].str.contains("Regular", case=False, na=False)).astype(int)
else:
    print("No 'gameType' column found — assigning default values.")
    final_df["is_playoffs"] = 0
    final_df["is_regular"] = 0


# --- 2. Add contextual variables such as attendance (if exists) ---
context_cols = [
    "home_win",
    "score_diff",
    "is_playoffs",
    "is_regular",
]

attendance_cols = [c for c in final_df.columns if "attendance" in c and "home" in c]
if attendance_cols:
    final_df["home_attendance"] = final_df[attendance_cols[0]]
    context_cols.append("home_attendance")
    print("Attendance column added.")
else:
    print("No home attendance column found — skipping.")


# --- 3. Confirm contextual feature creation ---
print(f"\nContextual features added: {len(context_cols)}")
print(f"Included columns: {context_cols}")
display(final_df[context_cols].head())

No 'gameType' column found — assigning default values.
No home attendance column found — skipping.

Contextual features added: 4
Included columns: ['home_win', 'score_diff', 'is_playoffs', 'is_regular']


,home_win,score_diff,is_playoffs,is_regular
0,1,19.0,0,0
1,1,10.0,0,0
2,1,6.0,0,0
3,1,27.0,0,0
4,0,-5.0,0,0


In [88]:
# ===============================================================
# 9. Rolling performance indicators (momentum features)
# ===============================================================
# Adds rolling 5-game averages for each team to capture performance trends.

def add_rolling_features(df, team_col, feature_cols, window=5):
    """
    Compute rolling averages for selected statistical features by team.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe containing per-game stats.
    team_col : str
        Column identifying the team (e.g., 'home_teamName').
    feature_cols : list
        List of columns to compute rolling averages for.
    window : int, default=5
        Number of previous games to include in the moving average.
    
    Returns
    -------
    df : pd.DataFrame
        Dataframe with new rolling average columns added.
    """
    # Identify a game date column dynamically
    date_cols = [c for c in df.columns if "gameDate" in c]
    if not date_cols:
        raise KeyError("No date column found to compute rolling features.")
    date_col = date_cols[0]

    # Ensure chronological order
    df = df.sort_values(by=[team_col, date_col])

    # Compute rolling means for each feature
    for col in feature_cols:
        if col not in df.columns:
            print(f"⚠️ Skipping missing column: {col}")
            continue

        new_col = f"{col}_rolling{window}"
        df[new_col] = (
            df.groupby(team_col)[col]
              .transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
        )

    return df


# --- Apply rolling averages for both home and away teams ---
window = 5  # rolling window size (define it here once)

rolling_features = [
    "home_points", "home_assists", "home_reboundsTotal",
    "home_fieldGoalsPercentage", "home_threePointersPercentage", "home_freeThrowsPercentage",
    "away_points", "away_assists", "away_reboundsTotal",
    "away_fieldGoalsPercentage", "away_threePointersPercentage", "away_freeThrowsPercentage"
]

final_df = add_rolling_features(final_df, team_col="home_teamName", feature_cols=rolling_features, window=5)
final_df = add_rolling_features(final_df, team_col="away_teamName", feature_cols=rolling_features, window=5)

print(f"Added rolling performance features ({len(rolling_features)} base columns × 2 teams).")
print(f"Rolling window size: {window} games.")
print(f"New rolling columns created: {[f'{col}_rolling5' for col in rolling_features]}")

Added rolling performance features (12 base columns × 2 teams).
Rolling window size: 5 games.
New rolling columns created: ['home_points_rolling5', 'home_assists_rolling5', 'home_reboundsTotal_rolling5', 'home_fieldGoalsPercentage_rolling5', 'home_threePointersPercentage_rolling5', 'home_freeThrowsPercentage_rolling5', 'away_points_rolling5', 'away_assists_rolling5', 'away_reboundsTotal_rolling5', 'away_fieldGoalsPercentage_rolling5', 'away_threePointersPercentage_rolling5', 'away_freeThrowsPercentage_rolling5']


In [104]:
# ===============================================================
# 10. Merge full game schedule to recover missing dates
# ===============================================================

import pandas as pd

# 1. Load official Games.csv
games_df = pd.read_csv("../data/raw/Games.csv", usecols=["gameId", "gameDate"])
games_df["gameId"] = pd.to_numeric(games_df["gameId"], errors="coerce").astype("Int64")
games_df["gameDate"] = pd.to_datetime(games_df["gameDate"], errors="coerce", utc=True)

print("Loaded Games.csv:", len(games_df), "rows")
print("Non-null gameDate:", games_df["gameDate"].notna().sum())

# 2. Prepare final_df for merge
if "gameId" not in final_df.columns:
    raise KeyError("The final_df is missing 'gameId' column required for merging.")
final_df["gameId"] = pd.to_numeric(final_df["gameId"], errors="coerce").astype("Int64")

# Remove any existing 'gameDate' column to avoid conflicts
if "gameDate" in final_df.columns:
    final_df.drop(columns=["gameDate"], inplace=True)
    print("Removed existing 'gameDate' column before merging.")

# 3. Merge using gameId
merged = final_df.merge(games_df, how="left", on="gameId", indicator=True)
print("\nMerge indicator summary:")
print(merged["_merge"].value_counts())

# 4. Fill missing home_gameDate values
if "home_gameDate" not in merged.columns:
    merged["home_gameDate"] = pd.NaT

mask_fill = merged["home_gameDate"].isna() & merged["gameDate"].notna()
filled_count = mask_fill.sum()

merged.loc[mask_fill, "home_gameDate"] = merged.loc[mask_fill, "gameDate"]
print(f"\nFilled {filled_count} missing home_gameDate values from Games.csv")

# Cleanup
merged.drop(columns=["gameDate", "_merge"], inplace=True, errors="ignore")

# 5. Validation
missing_after = merged["home_gameDate"].isna().sum()
print(f"Missing home_gameDate after merge: {missing_after}")

# 6. Preview sample
print("\nSample of merged rows:")
display(merged[["home_teamName", "away_teamName", "home_gameDate"]].sample(5, random_state=42))

# Update reference
final_df = merged

Loaded Games.csv: 72008 rows
Non-null gameDate: 129

Merge indicator summary:
_merge
both          72008
left_only         0
right_only        0
Name: count, dtype: int64

Filled 0 missing home_gameDate values from Games.csv
Missing home_gameDate after merge: 71789

Sample of merged rows:


,home_teamName,away_teamName,home_gameDate
43117,Warriors,Nets,NaT
43380,Bulls,Nuggets,NaT
5466,Pistons,Bucks,NaT
3826,Bullets,Bucks,NaT
34375,Grizzlies,Lakers,NaT


In [105]:
# ===============================================================
# 11. Cleanup, export, and validation
# ===============================================================

import numpy as np

# 1. Identify rolling columns
rolling_cols = [c for c in final_df.columns if "_rolling" in c]

# 2. Define export structure
base_cols = [
    "home_win", "score_diff",
    "diff_points", "diff_assists", "diff_reboundsTotal",
    "diff_steals", "diff_blocks",
    "diff_fieldGoalsPercentage", "diff_threePointersPercentage", "diff_freeThrowsPercentage",
    "is_playoffs", "is_regular",
    "home_teamName", "away_teamName", "home_gameDate"
]
export_cols = base_cols + rolling_cols

# 3. Filter to existing columns
available_cols = [c for c in export_cols if c in final_df.columns]
final_dataset = final_df[available_cols].copy()

# 4. Handle missing values
numeric_cols = final_dataset.select_dtypes(include=[np.number]).columns
final_dataset[numeric_cols] = final_dataset[numeric_cols].fillna(final_dataset[numeric_cols].median())

# 5. Sort chronologically
if "home_gameDate" in final_dataset.columns:
    final_dataset["home_gameDate"] = pd.to_datetime(final_dataset["home_gameDate"], errors="coerce")
    final_dataset = final_dataset.sort_values(by="home_gameDate").reset_index(drop=True)

# 6. Validation summary
print("Final dataset ready for export.")
print(f"Shape: {final_dataset.shape}")
print(f"Columns included: {len(final_dataset.columns)}")
print(f"Rolling features detected: {len(rolling_cols)}")

# 7. Missing-value summary
na_summary = final_dataset.isna().sum()
if na_summary.sum() > 0:
    print("\nColumns with missing values:")
    print(na_summary[na_summary > 0])
else:
    print("\nNo missing values detected.")

# 8. Export
output_path = "../data/final/final_dataset.csv"
final_dataset.to_csv(output_path, index=False)
print(f"\nDataset exported successfully to: {output_path}")

# 9. Sample preview
print("\nSample of exported data:")
display(final_dataset.sample(5, random_state=42))

Final dataset ready for export.
Shape: (72008, 27)
Columns included: 27
Rolling features detected: 12

Columns with missing values:
home_gameDate    71789
dtype: int64

Dataset exported successfully to: ../data/final/final_dataset.csv

Sample of exported data:


,home_win,score_diff,diff_points,diff_assists,diff_reboundsTotal,diff_steals,diff_blocks,diff_fieldGoalsPercentage,diff_threePointersPercentage,diff_freeThrowsPercentage,is_playoffs,is_regular,home_teamName,away_teamName,home_gameDate,home_points_rolling5,home_assists_rolling5,home_reboundsTotal_rolling5,home_fieldGoalsPercentage_rolling5,home_threePointersPercentage_rolling5,home_freeThrowsPercentage_rolling5,away_points_rolling5,away_assists_rolling5,away_reboundsTotal_rolling5,away_fieldGoalsPercentage_rolling5,away_threePointersPercentage_rolling5,away_freeThrowsPercentage_rolling5
43117,1,14.0,14.0,-2.0,6.0,6.0,0.0,-0.025,-0.110,0.085,0,0,Timberwolves,Nets,NaT,115.4,22.0,47.8,0.4470,0.3472,0.7610,118.4,26.0,50.2,0.4460,0.3876,0.8092
43380,0,-7.0,-7.0,-4.0,-9.0,-6.0,1.0,0.018,0.278,0.112,0,0,Bucks,Nuggets,NaT,101.6,18.2,43.8,0.4190,0.3106,0.7932,105.2,19.0,43.0,0.4540,0.3246,0.8114
5466,0,-2.0,-2.0,-6.0,-8.0,-1.0,0.0,-0.029,-0.006,-0.005,0,0,Pacers,Bucks,NaT,98.8,18.6,50.0,0.4476,0.2670,0.7504,89.8,16.0,44.6,0.3832,0.2818,0.7184
3826,1,13.0,13.0,5.0,-9.0,3.0,0.0,0.017,0.217,0.083,0,0,76ers,Bucks,NaT,98.4,19.6,45.2,0.4738,0.2576,0.7196,85.0,18.2,37.0,0.4226,0.2768,0.7704
34375,0,-11.0,-11.0,0.0,0.0,0.0,1.0,-0.071,0.004,-0.066,0,0,Celtics,Lakers,NaT,110.0,24.0,43.6,0.4668,0.3514,0.7580,99.4,22.2,42.0,0.4550,0.3430,0.7548
